Read the libraries

In [110]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline
from tqdm import tqdm_notebook
np.random.seed(500)
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter(action='ignore', category=ConvergenceWarning)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(ngram_range=(1,3))
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.svm import LinearSVC
import nltk
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lippd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Loading data and sample them for test nad training sets
df_test = pd.read_csv("/Users/lippd/Desktop/ML in R/Python/Cars 0-1/data/cepik_dane_testowe.csv")
df_train = pd.read_csv("/Users/lippd/Desktop/ML in R/Python/Cars 0-1/data/cepik_dane_uczące.csv")

In [ ]:
# Swaping missiing values in column "Type" to "---"
df_train['Type'] = df_train['Type'].replace(np.nan, '---')
df_test['Type'] = df_test['Type'].replace(np.nan, '---')

In [ ]:
# Connecting three column to one 
df_train['combined']=df_train['Mark'].astype(str)+' '+df_train['Model']+' '+df_train['Type']
df_test['combined']=df_test['Mark'].astype(str)+' '+df_test['Model']+' '+df_test['Type']

In [ ]:
df_train.head()

In [ ]:
df_train['combined'].values.astype('U')
df_test['combined'].values.astype('U')

In [ ]:
# Presenting the data
sns.countplot(y = "IsDeleted", data = df_train)

In [ ]:
sns.countplot(y = "Model", data = df_train)

In [ ]:
sns.countplot(y = "Mark", data = df_train)

In [ ]:
count_vect = CountVectorizer()

In [ ]:
# Creating bag of words
X_train_counts = count_vect.fit_transform(df_train.combined)

In [ ]:
print(count_vect.get_feature_names())

In [ ]:
X_train_counts.shape

In [ ]:
count_vect.vocabulary_.get(u'algorithm')

In [ ]:
tf_transform = TfidfTransformer(use_idf = False).fit(X_train_counts)

In [ ]:
# Transform to bag of words
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
X_train_tfidf.shape

In [ ]:
# Creating clasiffication model
clf = MultinomialNB().fit(X_train_tfidf, df_train.IsDeleted)

In [ ]:
X_new_counts = count_vect.transform(df_test.combined)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [ ]:
predicted = clf.predict(X_new_tfidf)

In [ ]:
df_predicted = pd.DataFrame(data = predicted)

In [ ]:
df_predicted.columns = ['results']

In [ ]:
df_predicted[df_predicted.results  == 1]

In [ ]:
for doc, combined in zip(df_test.combined, predicted):
    print('%r => %s' % (doc, df_train.combined))

# MultiNB algorythm

In [ ]:
# Creatin pipeline for MultinomailNB
text_clf_MultiNB = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [ ]:
np.mean(predicted == df_test.IsDeleted)

In [ ]:
# Compare calculating algorythm
print(metrics.classification_report(df_test.IsDeleted, predicted, target_names = ['yes', 'no']))

# SGDC Algorythm

In [ ]:
#Creating pipeline for SVM
text_clf_SVM = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss = 'hinge', penalty = 'l2',
                alpha = 1e-3, random_state = 42,
                max_iter = 5, tol = None)),
])

In [ ]:
text_clf_SVM.fit(df_train.combined, df_train.IsDeleted)

In [ ]:
predicted_SVM = text_clf_SVM.predict(df_test.combined)

In [ ]:
np.mean(predicted_SVM == df_test.IsDeleted)

In [ ]:
# Check results from SGDC algorythm
print(metrics.classification_report(df_test.IsDeleted, predicted_SVM, target_names = ['yes', 'no']))

In [ ]:
metrics.confusion_matrix(df_test.IsDeleted, predicted)

In [ ]:
parametrs = {
    'vect__ngram_range': [(1,1), (1,2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': [0.001, 0.0001, 0.00001, 0.000001],
}

In [ ]:
text_clf_SVM.get_params().keys()

In [ ]:
gs_clf = GridSearchCV(text_clf_SVM, parametrs, cv = 5, n_jobs = -1)
gs_clf = gs_clf.fit(df_train.combined, df_train.IsDeleted)

In [ ]:
df_train.IsDeleted[gs_clf.predict(df_test.combined)][1]

In [ ]:
gs_clf.best_score_

In [ ]:
for param_name in sorted(parametrs.keys()):
    print("%s: %r" % (param_name, 
                      gs_clf.best_params_[param_name]))

In [108]:
# checking results: after 
predicted_gs = gs_clf.predict(df_test.combined)
print(metrics.classification_report(df_test.IsDeleted, predicted_gs, target_names = ['yes', 'no']))
metrics.confusion_matrix(df_test.IsDeleted, predicted_gs)

              precision    recall  f1-score   support

         yes       0.98      1.00      0.99      6884
          no       0.99      0.61      0.76       311

    accuracy                           0.98      7195
   macro avg       0.99      0.81      0.87      7195
weighted avg       0.98      0.98      0.98      7195



array([[6882,    2],
       [ 120,  191]], dtype=int64)

# Logistic regression

In [117]:
clf_log = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)

In [ ]:
parametrs = {
    'C': [(1,1), (1,2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': [0.001, 0.0001, 0.00001, 0.000001],
}

In [113]:
x_train = df_train.combined
y_train = df_train.IsDeleted

In [116]:
df_train.combined

0                         SKODA FABIA 5J
1                             OPEL C ---
2                         FORD FOCUS DYB
3             RENAULT LAGUNA 1.9 DCI ---
4                            SEAT MII AA
                      ...               
8814    PORSCHE 991 CARRERA 4S COUPE ---
8815                    AUDI A6 2.0T ---
8816                 BMW MINI COOPER ---
8817             MERCEDES-BENZ VANEO ---
8818      DAIMLERCHRYSLER PT CRUISER ---
Name: combined, Length: 8819, dtype: object

In [ ]:
sentences = df_train['combined'].values.astype('U')
y = df_train['IsDeleted'].values

In [ ]:
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size = 0.25, random_state = 1000)

In [ ]:
sentences_train = df_train['combined']
sentences_test = df_test['combined']

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

In [ ]:
X_train = vectorizer.transform(sentences_train)
X_test = vectorizer.transform(sentences_test)

In [ ]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print('Accuracy for:{:.4f}', format(score))

In [ ]:
y_predicted = classifier.predict(df_test)